In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/covenant_wmi_remote_event_subscription_ActiveScriptEventConsumers.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','EventID','EventType','Message']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 20)
    & (df['Message'].str.lower().str.contains('.*type: script.*', regex=True))
]
)

,@timestamp,Hostname,EventID,EventType,Message
157,2020-09-02T01:43:50.916Z,WORKSTATION6.theshire.local,20,WmiConsumerEvent,WmiEventConsumer activity detected:\r\nRuleNam...
3307,2020-09-02T01:44:41.461Z,WORKSTATION6.theshire.local,20,WmiConsumerEvent,WmiEventConsumer activity detected:\r\nRuleNam...


In [4]:
(
df[['@timestamp','Hostname','EventID','SourceName','Message']]

[(df['Channel'] == 'Microsoft-Windows-WMI-Activity/Operational')
    & (df['EventID'] == 5861)
    & (df['Message'].str.lower().str.contains('.*scriptingengine = "vbscript".*', regex=True))
]
)

,@timestamp,Hostname,EventID,SourceName,Message
156,2020-09-02T01:43:50.913Z,WORKSTATION6.theshire.local,5861,Microsoft-Windows-WMI-Activity,Namespace = //./root/subscription; Eventfilter...


In [5]:
(
df[['@timestamp','Hostname','ParentImage','Image','CommandLine','ProcessId','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 1)
    & (df['Image'].str.lower().str.contains('.*scrcons.*', regex=True))
]
)

,@timestamp,Hostname,ParentImage,Image,CommandLine,ProcessId,ProcessGuid
1532,2020-09-02T01:44:10.608Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,972,{c4e35f20-f8ea-5f4e-c504-000000000400}


In [6]:
(
df[['@timestamp','Hostname','ParentProcessName','NewProcessName','CommandLine','NewProcessId']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4688)
    & (df['NewProcessName'].str.lower().str.contains('.*scrcons.*', regex=True))
]
)

,@timestamp,Hostname,ParentProcessName,NewProcessName,CommandLine,NewProcessId
1744,2020-09-02T01:44:11.705Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,0x3cc


In [7]:
(
df[['@timestamp','Hostname','Image','ImageLoaded','Description','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (df['ImageLoaded'].str.lower().isin(['c:\\windows\\system32\\wbem\\scrcons.exe',
        'c:\\windows\\system32\\\vbscript.dll',
        'c:\\windows\\system32\\wbem\\\wbemdisp.dll',
        'c:\\windows\\system32\\wshom.ocx',
        'c:\\windows\\system32\\scrrun.dll'
    ]))
]
)

,@timestamp,Hostname,Image,ImageLoaded,Description,ProcessGuid
1537,2020-09-02T01:44:10.611Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\wbem\scrcons.exe,WMI Standard Event Consumer - scripting,{c4e35f20-f8ea-5f4e-c504-000000000400}
1591,2020-09-02T01:44:11.640Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\wshom.ocx,Windows Script Host Runtime Library,{c4e35f20-f8ea-5f4e-c504-000000000400}
1600,2020-09-02T01:44:11.645Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\scrrun.dll,Microsoft ® Script Runtime,{c4e35f20-f8ea-5f4e-c504-000000000400}


In [8]:
imageLoadDf = (
# (
df[['@timestamp','Hostname','Image','ImageLoaded','Description','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (df['ImageLoaded'].str.lower().isin(['c:\\windows\\system32\\wbem\\scrcons.exe',
        'c:\\windows\\system32\\\vbscript.dll',
        'c:\\windows\\system32\\wbem\\\wbemdisp.dll',
        'c:\\windows\\system32\\wshom.ocx',
        'c:\\windows\\system32\\scrrun.dll'
    ]))
]
)

processCreateDf = (
# (
df[['@timestamp','Hostname','ParentImage','Image','CommandLine','ProcessId','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 1)
    & (df['Image'].str.lower().str.contains('.*scrcons.*', regex=True))
]
)

firstJoinDf = (
pd.merge(imageLoadDf, processCreateDf,
    on = 'ProcessGuid', how = 'inner')
)

networkConnectionDf = (
# (
df[['@timestamp','Hostname','Image','DestinationIp','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 3)
]
)

(
pd.merge(firstJoinDf, networkConnectionDf,
    on = 'ProcessGuid', how = 'inner')
)

,@timestamp_x,Hostname_x,Image_x,ImageLoaded,Description,ProcessGuid,@timestamp_y,Hostname_y,ParentImage,Image_y,CommandLine,ProcessId,@timestamp,Hostname,Image,DestinationIp
0,2020-09-02T01:44:10.611Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\wbem\scrcons.exe,WMI Standard Event Consumer - scripting,{c4e35f20-f8ea-5f4e-c504-000000000400},2020-09-02T01:44:10.608Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,972,2020-09-02T01:44:15.877Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,10.10.10.5
1,2020-09-02T01:44:11.640Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\wshom.ocx,Windows Script Host Runtime Library,{c4e35f20-f8ea-5f4e-c504-000000000400},2020-09-02T01:44:10.608Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,972,2020-09-02T01:44:15.877Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,10.10.10.5
2,2020-09-02T01:44:11.645Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\scrrun.dll,Microsoft ® Script Runtime,{c4e35f20-f8ea-5f4e-c504-000000000400},2020-09-02T01:44:10.608Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,972,2020-09-02T01:44:15.877Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,10.10.10.5


In [18]:
processCreateDf = (
# (
df[['@timestamp','Hostname','ParentImage','Image','CommandLine','ProcessId','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 1)
    & (df['Image'].str.lower().str.contains('.*scrcons.*', regex=True))
]
)

imageLoadDf = (
# (
df[['@timestamp','Hostname','Image','ImageLoaded','Description','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (df['ImageLoaded'].str.lower().isin(['c:\\windows\\system32\\wbem\\scrcons.exe',
        'c:\\windows\\system32\\\vbscript.dll',
        'c:\\windows\\system32\\wbem\\\wbemdisp.dll',
        'c:\\windows\\system32\\wshom.ocx',
        'c:\\windows\\system32\\scrrun.dll'
    ]))
]
)

firstJoinDf = (
pd.merge(processCreateDf, imageLoadDf,
    on = 'ProcessGuid', how = 'inner')
)

firstJoinDf['ProcessId'] = firstJoinDf['ProcessId'].apply(int).apply( hex )

networkLogonDf = (
# (
df[['@timestamp','Hostname','TargetUserName','TargetLogonId','IpAddress','ProcessId']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4624)
    & (df['LogonType'] == 3)
]
)

(
pd.merge(firstJoinDf, networkLogonDf,
    on = 'ProcessId', how = 'inner')
)

,@timestamp_x,Hostname_x,ParentImage,Image_x,CommandLine,ProcessId,ProcessGuid,@timestamp_y,Hostname_y,Image_y,ImageLoaded,Description,@timestamp,Hostname,TargetUserName,TargetLogonId,IpAddress
0,2020-09-02T01:44:10.608Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,0x3cc,{c4e35f20-f8ea-5f4e-c504-000000000400},2020-09-02T01:44:10.611Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\wbem\scrcons.exe,WMI Standard Event Consumer - scripting,2020-09-02T01:44:11.726Z,WORKSTATION6.theshire.local,pgustavo,0x11f0fcd,-
1,2020-09-02T01:44:10.608Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,0x3cc,{c4e35f20-f8ea-5f4e-c504-000000000400},2020-09-02T01:44:11.640Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\wshom.ocx,Windows Script Host Runtime Library,2020-09-02T01:44:11.726Z,WORKSTATION6.theshire.local,pgustavo,0x11f0fcd,-
2,2020-09-02T01:44:10.608Z,WORKSTATION6.theshire.local,C:\Windows\System32\svchost.exe,C:\Windows\System32\wbem\scrcons.exe,C:\windows\system32\wbem\scrcons.exe -Embedding,0x3cc,{c4e35f20-f8ea-5f4e-c504-000000000400},2020-09-02T01:44:11.645Z,WORKSTATION6.theshire.local,C:\Windows\System32\wbem\scrcons.exe,C:\Windows\System32\scrrun.dll,Microsoft ® Script Runtime,2020-09-02T01:44:11.726Z,WORKSTATION6.theshire.local,pgustavo,0x11f0fcd,-


In [10]:
(
df[['@timestamp','Hostname','TargetUserName','ImpersonationLevel','LogonType','ProcessName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4624)
    & (df['LogonType'] == 3)
    & (df['ProcessName'].str.lower().str.endswith('scrcons.exe', na=False))
]
)

,@timestamp,Hostname,TargetUserName,ImpersonationLevel,LogonType,ProcessName
1799,2020-09-02T01:44:11.726Z,WORKSTATION6.theshire.local,pgustavo,%%1832,3.0,C:\Windows\System32\wbem\scrcons.exe
